# **IMPORT LIBRARY FILES**

In [ ]:
pip install tsai[extras]

In [ ]:
!git clone https://github.com/fastai/fastai
!pip install -e "fastai[dev]"

In [ ]:
# **************** UNCOMMENT AND RUN THIS CELL IF YOU NEED TO INSTALL/ UPGRADE TSAI ****************
stable = False # Set to True for latest pip version or False for main branch in GitHub
!pip install {"tsai -U" if stable else "git+https://github.com/timeseriesAI/tsai.git"} >> /dev/null

In [ ]:
import tsai
import fastai
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ast
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
### Set float to 2 dec

pd.set_option('precision', 2)

In [ ]:
from tsai.all import *

In [ ]:
__all__ = ['TimeSplitter', 'RandomSplitter', 'check_overlap', 'check_splits_overlap', 'leakage_finder', 'balance_idx',
           'TrainValidTestSplitter', 'plot_splits', 'get_splits', 'get_walk_forward_splits', 'TSSplitter',
           'get_predefined_splits', 'combine_split_data', 'get_splits_len']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
### Set path to the correct folder

path = (r'/content/drive/MyDrive/ColabNotebooks')

In [ ]:
cd /content/drive/MyDrive/ColabNotebooks

# **Open Files Create DataFrame**

In [ ]:
### Every model (9 models) needs to be modelled serperately
### --------------------------------------------------------------------------

# 1) 
# df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Honda_Clarity_Electric/Honda_Clarity_100.csv")

#2)
# df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Chevy_Bolt/ChevyBolt_100.csv")

# 3)
# df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/VW_eGolf/VW_100.csv")

# 4)
# df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Tesla_Model_3/Tesla_100.csv")

# 5)
# df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Nissan_Leaf/Nissan_100.csv")

# 6)
# df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Kia_Soul/KiaSoul_100.csv")

# 7)
# df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Kia_e-Niro/KiaEniro_100.csv")

# 8)
# df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Hyundai_Kona_Electric/HyundaiKonaElectric_100.csv")

# 9)
# df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Ford_Focus/FordFocus_100.csv")

In [25]:
# COUNT ROWS IN FILE
# TO SPLIT INTO EQUAL SIZES

import pandas as pd

# count rows in a file
file = open("../data/september_ev.csv")
numline = len(file.readlines())
print (numline)

907206


In [26]:
# NECESSARY FOR GITHUB REPO
# GitHub filesize limit
 
for i,chunk in enumerate(pd.read_csv('../data/september_ev.csv', chunksize=302402)): 
    chunk.to_csv('../data/september_{}.csv'.format(i), index=False)

In [ ]:
### DF ANALYSIS - ROWS AND COLUMNS
### -----------------------------------------------------------

df.shape

In [ ]:
### DF ANALYSIS - COLUMN DATA TYPES
### -----------------------------------------------------------

df.info()

In [ ]:
df.columns

In [ ]:
### Create 70% column
### -----------------------------------------------------------

content = 0

df.insert(7, '70_battery', content)

In [ ]:
### Create 100% column
### -----------------------------------------------------------

content = 0

df.insert(8, '100_battery', content)

In [ ]:
### Save dataframe pre padding data frame to csv file
### -----------------------------------------------------------

df.to_csv("/content/drive/MyDrive/ColabNotebooks/Chevy_Bolt/ChevyBolt_latest.csv", index=False)

In [ ]:
### Charge Station dataset to variable - border
### -----------------------------------------------------------

border = pd.read_excel("ChargeStations.xlsx", sheet_name="BorderCommunities", header=None)

In [ ]:
### change start and end datetime to correct data types
### -----------------------------------------------------------

df['start_datetime'] = pd.to_datetime(df['start_datetime'])
df['end_datetime'] = pd.to_datetime(df['end_datetime'])

In [ ]:
### Feature engineer day, month, hour, and duration of trip
### -----------------------------------------------------------

df["start_daymonth"] = df["start_datetime"].dt.month
df["start_dayday"] = df["start_datetime"].dt.day
df["start_dayhour"] = df["start_datetime"].dt.hour

df["end_daymonth"] = df["end_datetime"].dt.month
df["end_dayday"] = df["end_datetime"].dt.day
df["end_dayhour"] = df["end_datetime"].dt.hour

df["duration"] = (df["end_datetime"] - df["start_datetime"]).dt.seconds

In [ ]:
### DF analysis
### -----------------------------------------------------------

df.columns

In [ ]:
### Drop columns
### -----------------------------------------------------------

to_remove = ["ev_model", "day", 'period']
df.drop(to_remove, axis=1, inplace=True)

In [ ]:
### Set border column variables
### -----------------------------------------------------------

border.columns = ["dropoff", "b1","b2","b3","b4","b5","b6","b7","b8"]

In [ ]:
### Check border head
### -----------------------------------------------------------

border.head()

In [ ]:
### Merge the model data and the boundary data into one dataframe
### -----------------------------------------------------------

df = df.merge(border, how='left')

In [ ]:
### Check the dataframe
### -----------------------------------------------------------

df.head(50)

In [ ]:
### Save dataframe pre padding data frame to csv file
### -----------------------------------------------------------

df.to_csv("/content/drive/MyDrive/ColabNotebooks/VW_eGolf/VW_latest.csv", index=False)

In [ ]:
### Replace NaN with 0
### -----------------------------------------------------------

df.fillna(0, inplace=True)

In [ ]:
### Check data types 
### -----------------------------------------------------------

df.info()

In [ ]:
### Using a dictionary convert border features from float64 to int64
### -----------------------------------------------------------

convert_dict = {
                'b1' : np.int64,
                'b2' : np.int64,
                'b3' : np.int64,
                'b4' : np.int64,
                'b5' : np.int64,
                'b6' : np.int64,
                'b7' : np.int64,
                'b8' : np.int64,

                }
df = df.astype(convert_dict)

# **Create Batch Id Column and batches by 20_battery col = 1.**

In [ ]:
### Use 20_battery = 1 to split dataframe into batches by batch_id
### copies batch_id to a batch id col
### Create list of batches
### -----------------------------------------------------------

batch_id = 0
batches = []
for ev_id in df['ev_id'].unique():
    df_ev_data = df[df['ev_id']==ev_id].sort_values("start_datetime").reset_index(drop=True)
    index_positions = df_ev_data[df_ev_data["20_battery"]==1].index
    start_pos = df_ev_data.index[0]
    for end_pos in index_positions:
        new_batch = df_ev_data.iloc[start_pos:end_pos,:].copy()
        new_batch["batch_id"] = batch_id
        batch_id = batch_id + 1
        batches.append(new_batch)
        start_pos = end_pos
    #last batch append
    final_batch = df_ev_data.iloc[end_pos:,:].copy()
    final_batch.loc[:,"batch_id"] = batch_id
    batch_id +=1
    batches.append(final_batch)

In [ ]:
### Check how many batches in df
### -----------------------------------------------------------

len(batches)

In [ ]:
### Concat batches into dataframe
### -----------------------------------------------------------

batch_df = pd.concat(batches)

In [ ]:
### Check df rows and columns
### -----------------------------------------------------------

batch_df.shape

In [ ]:
### ANALYSIS
### Check 20 batches are unique (must be unique)
### -----------------------------------------------------------

batch_df["batch_id"].value_counts().iloc[-20:]

In [ ]:
### Only select batches that has at least 20 data points
### Low value batches can creaste error when modelled
### -----------------------------------------------------------

batch_counts = batch_df["batch_id"].value_counts()
selected_batches = list(batch_counts[batch_counts>=20].index)
ver1_batch_df = batch_df[batch_df['batch_id'].isin(selected_batches)]

In [ ]:
### Check df rows and columns
### -----------------------------------------------------------

ver1_batch_df.shape

In [ ]:
### Check unique how many batch ids
### -----------------------------------------------------------

ver1_batch_df['batch_id'].nunique()

In [ ]:
### Check the dataframe column names
### -----------------------------------------------------------

ver1_batch_df.columns

# **Add Count to Batch ID in Preparation for Modelling**

In [ ]:
### Data Analysis
### -----------------------------------------------------------

ver1_batch_df.info()

In [ ]:
### Data Analysis
### -----------------------------------------------------------

ver1_batch_df.columns

In [ ]:
### GIVES ERROR BUT STILL WORKS?
### Group by batch id and add numeric count to batches
### -----------------------------------------------------------

ver1_batch_df["count"] = 1 + ver1_batch_df.groupby("batch_id").cumcount()

In [ ]:
### reset index rename new_df
### -----------------------------------------------------------

new_df = ver1_batch_df.reset_index(drop=True)

In [ ]:
new_df.head(50)

In [ ]:
### ANALYSIS
### count batches in df
### -----------------------------------------------------------

new_df['batch_id'].value_counts().max()

In [ ]:
### TEST REDUCING THIS VALUE ???
### Create list of batch_ids with unique counts
### -----------------------------------------------------------

df_s = new_df["batch_id"].value_counts() <= 1060
selected_batch = list(df_s[df_s].index)

In [ ]:
### Save list contents to dataframe
### -----------------------------------------------------------

df = new_df[new_df['batch_id'].isin(selected_batch)]

In [ ]:
### Save dataframe pre padding data frame to csv file
### -----------------------------------------------------------

df.to_csv("/content/drive/MyDrive/ColabNotebooks/VW_eGolf/batched.csv", index=False)

# **ADD PADDING TO BATCHES**

In [ ]:
### Analysis
### -----------------------------------------------------------

df.columns

In [ ]:
### GIVES ERROR BUT WORKS?
### DROP UNWANTED COLUMNS
### -----------------------------------------------------------

to_remove = ['ev_id', 'model_id', 'duration', 'start_datetime', 'end_datetime']
df.drop(to_remove, axis=1, inplace=True)

In [ ]:
### Add padding to index 0 
### -----------------------------------------------------------

samp = df.iloc[[0]].copy()
print(samp.columns)
for c in samp.columns:
    samp.loc[0,c] = 0

In [ ]:
### Analysis
### -----------------------------------------------------------

df.info()

In [ ]:
### Analysis
### -----------------------------------------------------------

samp

In [ ]:
### Analysis
### -----------------------------------------------------------

df['batch_id'].value_counts()

In [ ]:
### Analysis
### -----------------------------------------------------------

df.shape

In [ ]:
### Create times variable
### -----------------------------------------------------------

batch_data = df['batch_id']
times = len(batch_data)
times

In [ ]:
### Analysis
### -----------------------------------------------------------

df.info()

In [ ]:
### Checking df for negative values
### -----------------------------------------------------------

df[df < 0].sum()

In [ ]:
### Checking for negative values
### -----------------------------------------------------------

(df < 0).apply(lambda x: any(x))

In [ ]:
### Replace negative values with 0
### -----------------------------------------------------------

num = df._get_numeric_data()
num[num < 0] = 0

In [ ]:
### MAX LENGTH BATCH COUNT
### -----------------------------------------------------------

df['batch_id'].value_counts().max()

In [ ]:
### PADDING
### MUST USE MAX BATCH VALUE
### -----------------------------------------------------------

%%time
new_df = []
for batch in df['batch_id'].unique()[:]:
    to_append = []
    batch_data = df[df['batch_id']==batch].copy()
    times = 1060 - len(batch_data)
    if times!=0:
        samp_df = samp.loc[samp.index.repeat(times)].reset_index(drop=True)
        samp_df["batch_id"] = batch
        to_append.append(samp_df)
    to_append.append(batch_data)
    batch_df = pd.concat(to_append)
    new_df.append(batch_df)

In [ ]:
### Concat padded list save as df
### -----------------------------------------------------------

padded_df = pd.concat(new_df)

In [ ]:
### Analysis
### -----------------------------------------------------------

padded_df.shape

In [ ]:
### SAVE DF TO CSV FILE
### -----------------------------------------------------------

padded_df.to_csv("/content/drive/MyDrive/ColabNotebooks/Chevy_Bolt/padded.csv", index=False)